In [18]:
import pandas as pd
import numpy as np
import pickle

In [40]:
data = pd.read_excel('~/Desktop/test.xlsx')
data.head()

,pay_type_description,total_dollars,total_hours,hourly_rate
0,Paid Administrative Leave,45310.88,1257.86,36.022196
1,NaN,74601.36,3110.97,23.980096
2,Orient Red Reg,7482.63,149.00,50.218993
3,Holiday Worked Shift 10%,477505.04,7589.31,62.918110
4,NaN,44700.53,5959.99,7.500102


In [41]:
list(data.columns) == ['pay_type_description', 'total_dollars', 'total_hours', 'hourly_rate']

True

In [11]:
len(data)

12479

In [7]:
# Loading our ML Model
rf_use_hours = pickle.load(open("models/RandomForest_use_hours.pkl", "rb"))
rf_use_salaries = pickle.load(open("models/RandomForest_use_salaries.pkl", "rb"))
rf_is_productive = pickle.load(open("models/RandomForest_is_productive.pkl", "rb"))

# Loading the String Classifier
string_classifier = pickle.load(open("models/string_classifier.pkl", "rb"))

In [27]:
# Initialize Results
use_hours = []
use_salaries = []
is_productive = []

In [28]:
for index, row in data.iterrows():
    pay_type_description = str(row.pay_type_description).upper()
    
    if pay_type_description in string_classifier:
        info = string_classifier[pay_type_description]
        use_hours.append(info["use_hours"])
        use_salaries.append(info["use_salaries"])
        is_productive.append(info["is_productive"])
        
    else: 
        
        total_dollars = row.total_dollars
        total_hours = row.total_hours
        hourly_rate = row.hourly_rate
        
        features = np.array([total_dollars, total_hours, hourly_rate]) 
        features = features.reshape((1, -1))
        
        use_hours.append(round(rf_use_hours.predict(features)[0]))
        use_salaries.append(round(rf_use_salaries.predict(features)[0]))
        is_productive.append(round(rf_is_productive.predict(features)[0]))

In [32]:
data["use_hours"] = use_hours
data["use_salaries"] = use_salaries
data["is_productive"] = is_productive

In [33]:
data.head()

,pay_type_description,total_dollars,total_hours,hourly_rate,use_hours,use_salaries,is_productive
0,Paid Administrative Leave,45310.88,1257.86,36.022196,1.0,1.0,1.0
1,NaN,74601.36,3110.97,23.980096,1.0,1.0,1.0
2,Orient Red Reg,7482.63,149.00,50.218993,1.0,1.0,1.0
3,Holiday Worked Shift 10%,477505.04,7589.31,62.918110,1.0,1.0,1.0
4,NaN,44700.53,5959.99,7.500102,0.0,1.0,1.0


In [34]:
data.to_excel("~/Desktop/result.xlsx")